In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


Dans ce détecteur automatique, nous essaierons d'automatiser le processus de classification.
après les expériences précédentes, nous avons vu que la précision des modèles de perception du contexte dépasse le modèle général où le prix est compris entre 96 et 200 ([96,200]) mais la précision générale du modèle l'emporte lorsque le prix est compris entre 200 et 310 ([201,310]).
***
après cela, dans les expériences, nous avons tenté de combiner les deux premiers modèles de contexte avec le modèle général, mais la précision du modèle général était trop faible (68,08%) dans la catégorie de prix [201,310], nous avons donc essayé de diviser le jeu de données différemment.
***
et pour cela, nous avons créé une combinaison des trois premiers modèles d'approche de la prise de conscience du contexte et du modèle général et classer par l'un des modèles en fonction du coût du produit.
***
la deuxième division était la suivante:
1. coût du produit [96,150]: modèle de prise de conscience du contexte 1.
2. coût du produit [151 200]: modèle de prise de conscience du contexte 2.
3. coût du produit [201 250]: modèle de prise de conscience du contexte 3.
4. coût du produit [251,310]: modèle général.
***
La deuxième division nous a apporté de bonnes améliorations de précision, elle a surpassé le modèle général (de 73 à 75).
***

Les étapes pour créer une combinaison des modèles mentionnés ci-dessus sont les suivantes:
1. importer les quatre modèles et le jeu de données + prétraiter le jeu de données pour correspondre aux modèles.
2. choisissez au hasard une ligne à partir de celui-ci (au lieu de saisir les données manuellement)
3. Supprimez la colonne Reached On Time et extrayez le prix du produit de la ligne.
4. classer le produit dans l'un des modèles selon son prix.
5. imprimer si le produit est arrivé à temps ou non.

#1.  Importer les quatre modèles et le jeu de données + prétraiter le jeu de données pour qu'il corresponde aux modèles.

In [ ]:
CA_model1=pickle.load(open('/content/Drive/MyDrive/context awareness models/Shipment tracking models/DT/model 1 optimized acc_76_851.sav', 'rb'))
CA_model2=pickle.load(open('/content/Drive/MyDrive/context awareness models/Shipment tracking models/DT/model 2 optimized acc_75_219.sav', 'rb'))
CA_model3=pickle.load(open('/content/Drive/MyDrive/context awareness models/Shipment tracking models/DT/model 3 optimized acc_71_814.sav', 'rb'))
G_model=pickle.load(open('/content/Drive/MyDrive/e-comerce model/model3_DecisionTree_ecommerce_73-93.sav', 'rb'))

In [ ]:
df = pd.read_csv("/content/Drive/MyDrive/context awareness datasets/shipment tracking dataset/data.csv")


In [ ]:
df.drop(['ID'], axis=1, inplace=True)

##############################################################################################################

df_reached = df[df['Reached.on.Time_Y.N'] == 0]
df_not_reached = df[df['Reached.on.Time_Y.N'] == 1]

###############################################################################################################

df_reached_downsampled = resample(df_reached,replace=False,n_samples=4000,random_state=25) 
df_not_reached_downsampled = resample(df_not_reached,replace=False,n_samples=4000,random_state=25) 
df_downsampled = pd.concat([df_not_reached_downsampled,df_reached_downsampled])

###############################################################################################################

df_downsampled.Product_importance[df_downsampled.Product_importance == 'low'] = 1
df_downsampled.Product_importance[df_downsampled.Product_importance == 'medium'] = 2
df_downsampled.Product_importance[df_downsampled.Product_importance == 'high'] = 3

df_downsampled.Gender[df_downsampled.Gender == 'F'] = 0
df_downsampled.Gender[df_downsampled.Gender == 'M'] = 1

###############################################################################################################

df_encoded=pd.get_dummies(df_downsampled,columns=['Warehouse_block','Mode_of_Shipment'])

In [ ]:
df_encoded.head()

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship
2554,5,5,264,3,2,0,23,2837,1,0,0,1,0,0,0,0,1
2529,2,1,255,3,1,1,19,1551,1,0,1,0,0,0,0,0,1
1073,3,1,148,2,2,1,34,1053,1,0,0,0,0,1,0,1,0
9432,6,3,271,4,1,0,2,1364,1,0,0,0,1,0,0,1,0
2539,2,5,252,2,1,0,21,2070,1,0,0,0,0,1,0,0,1


#2. choisissez au hasard une ligne à partir de celui-ci (au lieu de saisir les données manuellement)  

In [ ]:
data_point = df_encoded.sample()
data_point

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship
2782,4,4,173,3,2,0,61,1609,1,0,0,1,0,0,0,0,1


#3. supprimez la colonne Reached On Time et extrayez le prix du produit de la ligne.

In [ ]:
X=data_point.drop("Reached.on.Time_Y.N", axis = 1)
price=X.iloc[0]['Cost_of_the_Product']
X

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship
2782,4,4,173,3,2,0,61,1609,0,0,1,0,0,0,0,1


In [ ]:
price

173

#4. classez le produit dans l'un des modèles en fonction de son prix.

In [ ]:
if (price <=150 ):
  prediction=CA_model1.predict(X)
elif ((price >150 ) & (price <=200)):
  prediction=CA_model2.predict(X)
elif ((price >200 ) & (price <=250)):
  prediction=CA_model3.predict(X)
elif (price >250 ):
  prediction=G_model.predict(X)
 


#5. imprimer si le produit est arrivé à temps ou non.

In [ ]:
if (prediction[0]==1) : 
  print("The product has NOT reached on time ✘")
else :
  print ("The product has reached on time ✔ ")

The product has NOT reached on time ✘
